## Check Buildstock sampling per 15 SF prototypes (Elevate Energy)

In [1]:
import pandas as pd
import numpy as np
import os
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files


### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [125]:
# local path to downloaded results
iteration = '_11' # <----- options: 'base', '_01'~'_11', '_national_2018'
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = f'cookcnty_2007{iteration}' 
baseline_result = os.path.join(result_path,iteration_path,'buildstock.csv')
print(f'Result and file storage path: {iteration_path}')


Result and file storage path: cookcnty_2007_11


In [126]:
res = pd.read_csv(baseline_result)
res

,Building,Location,Usage Level,Corridor,Vacancy Status,Eaves,Door Area,Overhangs,Interior Shading,Doors,...,Misc Freezer,Misc Hot Tub Spa,Misc Pool,Misc Pool Heater,Misc Pool Pump,HVAC Has Ducts,Ducts,Infiltration,Geometry Perimeter Footprint Ratio,Chicago
0,1,IL_Chicago-OHare.Intl.AP.725300,Medium,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, Uninsulated",15 ACH50,0.13,Non-City Cook County
1,2,IL_Chicago-OHare.Intl.AP.725300,High,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, Uninsulated",40 ACH50,0.13,City of Chicago
2,3,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,"EF 12, National Average",None,None,None,None,Yes,In Finished Space,8 ACH50,0.13,Non-City Cook County
3,4,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",15 ACH50,0.13,Non-City Cook County
4,5,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",15 ACH50,0.13,Non-City Cook County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, Uninsulated",25 ACH50,0.13,Non-City Cook County
39996,39997,IL_Chicago-OHare.Intl.AP.725300,Medium,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",25 ACH50,0.13,Non-City Cook County
39997,39998,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,None,None,None,None,None,Yes,"20% Leakage, R-4",25 ACH50,0.13,City of Chicago
39998,39999,IL_Chicago-OHare.Intl.AP.725300,Low,Double-Loaded Interior,Occupied,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,...,"EF 12, National Average",None,None,None,None,Yes,"30% Leakage, Uninsulated",5 ACH50,0.13,City of Chicago


In [127]:
if iteration == '_national_2018':
    res = res[res['AHS Region']=='CBSA Chicago-Naperville-Elgin, IL-IN-WI'].reset_index(drop=True)
    res

In [128]:
### HVAC cols changed after HVAC restructuring (implemented in Run5)
if iteration in ['base','_01','_02','_04']:
    cooling_col = 'HVAC System Cooling Type'
else:
    cooling_col = 'HVAC Cooling Type'
if iteration in ['base','_01','_02','_04']:
    heating_col = 'HVAC System Heating Natural Gas'
else:
    heating_col = 'HVAC Heating Type And Fuel'
print('selected')


selected


### Check housing charateristics distributions

In [129]:
proto_chars = ['Geometry Stories',
              'Geometry Wall Type',
              'Vintage ACS',
              cooling_col,
              heating_col,
              'Geometry Floor Area'
             ]

Ns = len(res);
print(f'>>> ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} samples ran created \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['Building'].count()
    Nchar = Nchar/Ns
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, iteration_path, 'buildstock_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, iteration_path)}')

>>> ResStock - cookcnty_2007_11 - RESULT SUMMARY:

  * 40000 samples ran created 

>>> Housing characteristics splits:

  * [1] Geometry Stories
1    0.52545
2    0.26075
3    0.21380
Name: Building, dtype: float64

  * [2] Geometry Wall Type
Masonry     0.71495
WoodStud    0.28505
Name: Building, dtype: float64

  * [3] Vintage ACS
1940-59    0.235025
1960-79    0.255075
1980-99    0.137875
2000-09    0.076125
2010s      0.011675
<1940      0.284225
Name: Building, dtype: float64

  * [4] HVAC Cooling Type
Central AC    0.61605
Heat Pump     0.02560
None          0.05250
Room AC       0.30585
Name: Building, dtype: float64

  * [5] HVAC Heating Type And Fuel
Electricity ASHP                       0.020250
Electricity Baseboard                  0.044775
Electricity Electric Boiler            0.000750
Electricity Electric Furnace           0.028950
Electricity Electric Wall Furnace      0.000900
Electricity Shared Heating             0.035150
Fuel Oil Fuel Boiler                   0.000

### Add additional metrics to ResStock results

In [130]:
# (1) add sqft and eui
sqft_map = {'0-499': 328,
            '500-749': 633,
            '750-999': 885,
            '1000-1499': 1220,
            '1500-1999': 1690,
            '2000-2499': 2176,
            '2500-2999': 2663,
            '3000-3999': 3301,
            '4000+': 8194}

res['sqft'] = res['Geometry Floor Area'].map(sqft_map)

## EE prototype tags
res['vintage_ee'] = 'post-1978'
res.loc[res['Vintage ACS'].isin(['1940-59','1960-79']),'vintage_ee'] = '1942-1978'
res.loc[res['Vintage ACS']=='<1940','vintage_ee'] = 'pre-1942'

res['stories'] = '2 or more stories'
res.loc[res['Geometry Stories']==1,'stories'] = 'less than 2 stories'

print('new cols added to "res"')

new cols added to "res"


In [131]:
### get sqft distr by building type and vintage
res_sqft_distr = res.groupby(['Geometry Building Type RECS', 'Vintage ACS',
    'Geometry Floor Area'])['Building'].count().to_frame()

# normalize
res_sqft_distr = res_sqft_distr.groupby(level = [0,1])['Building'].apply(lambda x: x/x.sum()).unstack(level=2)

# export
#res_sqft_distr.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_floor_area.csv'), index=True)
#print('sqft distr by vintage & bldg type exported')
res_sqft_distr

Geometry Floor Area                           0-499  1000-1499  1500-1999  \
Geometry Building Type RECS   Vintage ACS                                   
Mobile Home                   1940-59           NaN   1.000000        NaN   
                              1960-79           NaN   0.494505   0.175824   
                              1980-99           NaN   0.401961   0.196078   
                              2000-09           NaN   0.259259   0.259259   
                              2010s        0.125000   0.125000   0.125000   
                              <1940        0.105263   0.105263   0.105263   
Multi-Family with 2 - 4 Units 1940-59      0.025373   0.294776   0.224627   
                              1960-79      0.095652   0.334783        NaN   
                              1980-99      0.052270   0.374140   0.094911   
                              2000-09           NaN   0.451253   0.211699   
                              2010s             NaN   0.423077        NaN   
                              <1940        0.059256   0.473555   0.141283   
Multi-Family with 5+ Units    1940-59      0.080925   0.260116   0.081503   
                              1960-79      0.046192   0.266167   0.031211   
                              1980-99      0.101802   0.204054   0.062613   
                              2000-09      0.045662   0.264188   0.153947   
                              2010s        0.217712   0.158672        NaN   
                              <1940        0.215633   0.205596   0.056873   
Single-Family Attached        1940-59           NaN   1.000000        NaN   
                              1960-79           NaN   0.426877   0.239130   
                              1980-99           NaN   0.416883   0.264935   
                              2000-09           NaN   0.336182   0.336182   
                              2010s             NaN   0.250000   0.428571   
                              <1940        0.343434   0.303030   0.257576   
Single-Family Detached        1940-59      0.005005   0.326493   0.269436   
                              1960-79      0.002921   0.246012   0.326219   
                              1980-99           NaN   0.130307   0.228774   
                              2000-09      0.021390   0.054813   0.145722   
                              2010s             NaN   0.120370   0.120370   
                              <1940             NaN   0.254762   0.211376   

Geometry Floor Area                        2000-2499  2500-2999  3000-3999  \
Geometry Building Type RECS   Vintage ACS                                    
Mobile Home                   1940-59            NaN        NaN        NaN   
                              1960-79            NaN        NaN        NaN   
                              1980-99            NaN        NaN        NaN   
                              2000-09            NaN        NaN        NaN   
                              2010s         0.125000   0.125000   0.125000   
                              <1940         0.157895   0.105263   0.105263   
Multi-Family with 2 - 4 Units 1940-59       0.023134        NaN        NaN   
                              1960-79            NaN        NaN        NaN   
                              1980-99            NaN        NaN        NaN   
                              2000-09       0.089136        NaN        NaN   
                              2010s         0.192308        NaN        NaN   
                              <1940         0.033301   0.012733   0.026200   
Multi-Family with 5+ Units    1940-59            NaN        NaN   0.024277   
                              1960-79            NaN   0.008240        NaN   
                              1980-99       0.037838        NaN        NaN   
                              2000-09       0.133725        NaN        NaN   
                              2010s              NaN        NaN        NaN   
                              <1940         0.072689 

In [132]:
### get foundation type distr by building type, vintage (EE prototype)

res_found_distr = res[res['Geometry Building Type RECS'] =="Single-Family Detached"].groupby(
    ['vintage_ee', 'stories',
    'Geometry Foundation Type'])['Building'].count().to_frame()

# normalize
res_found_distr = res_found_distr.groupby(level = [0,1])['Building'].apply(lambda x: x/x.sum()).unstack(level=2)
res_found_distr


Geometry Foundation Type           Crawl  Heated Basement  Pier and Beam  \
vintage_ee stories                                                         
1942-1978  2 or more stories    0.138303         0.385090       0.022108   
           less than 2 stories  0.140118         0.399059       0.020235   
post-1978  2 or more stories    0.132862         0.355346       0.022799   
           less than 2 stories  0.156250         0.351562       0.024219   
pre-1942   2 or more stories    0.171842         0.262867       0.009783   
           less than 2 stories  0.152554         0.303009       0.015395   

Geometry Foundation Type            Slab  Unheated Basement  
vintage_ee stories                                           
1942-1978  2 or more stories    0.204113           0.250386  
           less than 2 stories  0.210118           0.230471  
post-1978  2 or more stories    0.272799           0.216195  
           less than 2 stories  0.286719           0.181250  
pre-1942   2 or more stories    0.074011           0.481497  
           less than 2 stories  0.071379           0.457663

In [133]:
res[res['Geometry Building Type RECS'] == "Single-Family Detached"].groupby([
    'stories','Geometry Wall Type',
'vintage_ee'])['sqft'].mean().unstack(level=-1)

vintage_ee                                1942-1978    post-1978     pre-1942
stories             Geometry Wall Type                                       
2 or more stories   Masonry             2388.954774  4054.416988  2549.022752
                    WoodStud            2265.897302  3614.343501  2322.710784
less than 2 stories Masonry             1680.132653  2386.103801  1460.060606
                    WoodStud            1690.260624  2243.300336  1402.300142

In [134]:
res[res['Geometry Building Type RECS'] == "Single-Family Detached"].groupby([
    'Geometry Wall Type','stories',
'Vintage'])['sqft'].mean().unstack(level=-1)

Vintage                                       1940s        1950s        1960s  \
Geometry Wall Type stories                                                      
Masonry            2 or more stories    2200.928571  2308.447293  2551.850220   
                   less than 2 stories  1614.780231  1595.706681  1815.888285   
WoodStud           2 or more stories    2110.318750  1979.318945  2512.986971   
                   less than 2 stories  1500.051351  1547.679330  1838.131034   

Vintage                                       1970s        1980s        1990s  \
Geometry Wall Type stories                                                      
Masonry            2 or more stories    2551.695652  3902.943089  3272.518519   
                   less than 2 stories  1903.677481  2049.713514  2147.518367   
WoodStud           2 or more stories    2524.535849  3398.944615  3400.850962   
                   less than 2 stories  1769.326112  2232.348659  2037.242991   

Vintage                                       2000s        2010s        <1940  
Geometry Wall Type stories                                                     
Masonry            2 or more stories    4415.254386  5364.312500  2549.022752  
                   less than 2 stories  2816.277533  3239.296296  1460.060606  
WoodStud           2 or more stories    4018.537234  4778.666667  2322.710784  
                   less than 2 stories  2589.076190  2908.812500  1402.300142

In [135]:
res_by_foundation = res[(res['Geometry Building Type RECS'] == "Single-Family Detached")].groupby([
    'vintage_ee', 'stories', 'Geometry Wall Type',
    'Geometry Foundation Type'
])

res_by_foundation3 = res_by_foundation['Building'].count().unstack(level=0)
res_by_foundation4 = res_by_foundation['sqft'].mean().unstack(level=0)

res_by_foundation3['type'] = 'count'
res_by_foundation4['type'] = 'sqft'

res_by_foundation_all = pd.concat([res_by_foundation3, res_by_foundation4], axis=0)
#res_by_foundation_all.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_foundation.csv'), index=True)
print('foundation type count, sqft by vintage & bldg type exported')

res_by_foundation_all.head(10*4)

foundation type count, sqft by vintage & bldg type exported


vintage_ee                                                         1942-1978  \
stories             Geometry Wall Type Geometry Foundation Type                
2 or more stories   Masonry            Crawl                       86.000000   
                                       Heated Basement            370.000000   
                                       Pier and Beam                8.000000   
                                       Slab                       157.000000   
                                       Unheated Basement          175.000000   
                    WoodStud           Crawl                      183.000000   
                                       Heated Basement            379.000000   
                                       Pier and Beam               35.000000   
                                       Slab                       240.000000   
                                       Unheated Basement          312.000000   
less than 2 stories Masonry            Crawl                      747.000000   
                                       Heated Basement           2397.000000   
                                       Pier and Beam               94.000000   
                                       Slab                      1122.000000   
                                       Unheated Basement         1128.000000   
                    WoodStud           Crawl                      444.000000   
                                       Heated Basement            995.000000   
                                       Pier and Beam               78.000000   
                                       Slab                       664.000000   
                                       Unheated Basement          831.000000   
2 or more stories   Masonry            Crawl                     2368.360465   
                                       Heated Basement           2413.175676   
                                       Pier and Beam             1916.375000   
                                       Slab                      2436.547771   
                                       Unheated Basement         2326.771429   
                    WoodStud           Crawl                     2109.306011   
                                       Heated Basement           2242.741425   
                                       Pier and Beam             2313.714286   
                                       Slab                      2292.420833   
                                       Unheated Basement         2360.105769   
less than 2 stories Masonry            Crawl                     1707.542169   
                                       Heated Basement           1654.535252   
                                       Pier and Beam             1550.797872   
                                       Slab                      1724.882353   
                                       Unheated Basement         1682.641844   
                    WoodStud           Crawl                     1674.292793   
                                       Heated Basement           1693.192965   
                                       Pier and Beam             1791.320513   
                                       Slab                      1696.394578   
                                       Unheated Basement         1680.894103   

vintage_ee                                                         post-1978  \
stories             Geometry Wall Type Geometry Foundation Type                
2 or more stories   Masonry            Crawl                       66.000000   
                                       Heated Basement            175.000000   
                                       Pier and Beam               15.000000   
                                       Slab                       169.000000   
                                       Unheated Basement           93.000000   
                    WoodStud           Crawl                      103.000000   

### Get results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags to ResStock results as prototypes overlap in chars

In [136]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))
print('prototypes loaded')


prototypes loaded


In [137]:
def get_res_by_prototype(res, row):
    res_group_i = res[res['Geometry Stories'].isin(row['Stories'].split(',')) & \
        res['Geometry Wall Type'].isin(row['WallType'].split(',')) & \
        res['Vintage ACS'].isin(row['Vintage'].split(',')) & \
        res['Geometry Building Type RECS'].isin(['Single-Family Detached'])]
    
    return res_group_i
    
    
### (3) get count, median & mean sqft
Metric_map1 = {'count': '',
               'median sqft': 'sqft'}
Metric_map2 = {'mean sqft': 'sqft'}

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN


for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.iloc[i, groups.columns.get_loc('count')] = \
                len(res_group_i)
        else:
            groups.iloc[i, groups.columns.get_loc(metric)] = \
                res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('>> median & mean sqft & eui computed:');

groups.set_index(['HousingGroupNo','HousingGroupName'])[{**Metric_map1, **Metric_map2}]


>> median & mean sqft & eui computed:


,,count,median sqft,mean sqft
HousingGroupNo,HousingGroupName,,,
1,Masonry Post-1978 less than 2 stories,684.0,2176.0,2386.103801
2,Masonry All Years Split Level,9135.0,1690.0,1999.694691
3,Masonry Post-1978 2 or more stories,518.0,3301.0,4054.416988
4,Masonry 1942-1978 Less than 2 Stories,5488.0,1690.0,1680.132653
5,Masonry 1942-1978 2 or more stories,796.0,2176.0,2388.954774
6,Masonry Pre-1942 less than 2 stories,726.0,1220.0,1460.060606
7,Masonry Pre-1942 2 or more stories,923.0,2176.0,2549.022752
8,Frame Post-1978 Less than 2 stories,596.0,2176.0,2243.300336
9,Frame Post-1978 Split Level,1350.0,2176.0,3009.053333


In [138]:
### (5) get heating & cooling and floor area char breakdowns

House = []
House_chars = [cooling_col, heating_col,
              'Infiltration',
              'Insulation Wall',
              'Insulation Unfinished Attic',
              'Insulation Unfinished Basement'
              ]

for i, row in groups.iterrows():
    
    res_group_i = get_res_by_prototype(res, row)
    
    Ns = len(res_group_i)
    Hi = []
    for char in House_chars:
        Nchar = res_group_i.groupby(char)['Building'].count()/Ns # fractions
        Nchar = Nchar.rename('fraction').to_frame()
        Nchar['housing_char'] = Nchar.index.name
        Hi.append(Nchar)
    
    Hi = pd.concat(Hi, axis=0)
    Hi.index.name = 'sub_char'
    Hi = Hi.reset_index()
    Hi['HousingGroupNo'] = row['HousingGroupNo']
    
    House.append(Hi)
    
House = pd.concat(House, axis=0)
House = pd.pivot_table(House, values='fraction', index=['HousingGroupNo'],
                    columns=['housing_char','sub_char'], aggfunc=np.sum)


print('computed')

computed


In [139]:
House['Insulation Wall']

pd.Series(insul_wall_dict)[House['Insulation Wall'].columns]

sub_char
Brick, 8-in, 2-wythe, Uninsulated    -1
CMU, 6-in Hollow, R-11               11
CMU, 6-in Hollow, R-19               19
CMU, 6-in Hollow, R-7                 7
CMU, 6-in Hollow, Uninsulated         0
Wood Stud, R-11                      11
Wood Stud, R-19                      19
Wood Stud, R-7                        7
Wood Stud, Uninsulated                0
dtype: int64

In [140]:
### calculate composite scores for infiltration, insulation wall, insulation unfinished attic:
infilt_dict = {
    '1 ACH50': 1,
    '10 ACH50': 10,
    '15 ACH50': 15,
    '2 ACH50': 2,
    '20 ACH50': 20,
    '25 ACH50': 25,
    '3 ACH50': 3,
    '30 ACH50': 30,
    '4 ACH50': 4,
    '40 ACH50': 40,
    '5 ACH50': 5,
    '50 ACH50': 50,
    '6 ACH50': 6,
    '7 ACH50': 7,
    '8 ACH50': 8
}

insul_wall_dict = {
    'Brick, 8-in, 2-wythe, Uninsulated': -1,
    'CMU, 6-in Hollow, R-11': 11,
    'CMU, 6-in Hollow, R-15': 15,
    'CMU, 6-in Hollow, R-19': 19,
    'CMU, 6-in Hollow, R-7': 7,
    'CMU, 6-in Hollow, Uninsulated': 0,
    'Wood Stud, R-11': 11,
    'Wood Stud, R-15': 15,
    'Wood Stud, R-19': 19,
    'Wood Stud, R-7': 7,
    'Wood Stud, Uninsulated': 0
}

insul_attic_dict = {
    'Ceiling R-13, Vented': 13,
    'Ceiling R-19, Vented': 19,
    'Ceiling R-30, Vented': 30,
    'Ceiling R-38, Vented': 38,
    'Ceiling R-49, Vented': 49,
    'Ceiling R-7, Vented': 7,
    'Uninsulated, Vented': 0
}

insul_basement_dict = {
    'Ceiling R-13': 13,
    'Ceiling R-19': 19,
    'None': 0,
    'Uninsulated': 0
}


for char, dic in zip(['Infiltration','Insulation Wall',
                      'Insulation Unfinished Attic','Insulation Unfinished Basement'],
                    [infilt_dict, insul_wall_dict, 
                     insul_attic_dict, insul_basement_dict]):
    
    House[(char,'score')] = np.matmul(House[char].fillna(0), 
                                       pd.Series(dic)[House[char].columns])

House

housing_char   HVAC Cooling Type                                \
sub_char              Central AC Heat Pump      None   Room AC   
HousingGroupNo                                                   
1                       0.972222  0.013158  0.004386  0.010234   
2                       0.835796  0.005802  0.010290  0.148112   
3                       0.961390  0.027027  0.007722  0.003861   
4                       0.863338  0.003462  0.011844  0.121356   
5                       0.868090  0.001256  0.012563  0.118090   
6                       0.650138  0.005510  0.009642  0.334711   
7                       0.618635  0.006501  0.005417  0.369447   
8                       0.953020  0.018456  0.011745  0.016779   
9                       0.958519  0.019259  0.006667  0.015556   
10                      0.962865  0.019894  0.002653  0.014589   
11                      0.885790  0.003652  0.014608  0.095950   
12                      0.616612  0.003285  0.006570  0.373534   
13                      0.874674  0.001741  0.017406  0.106179   
14                      0.624467  0.002845  0.007112  0.365576   
15                      0.612745  0.003501  0.006303  0.377451   

housing_char   HVAC Heating Type And Fuel                        \
sub_char                 Electricity ASHP Electricity Baseboard   
HousingGroupNo                                                    
1                                0.007310              0.014620   
2                                0.004817              0.022879   
3                                0.027027              0.009653   
4                                0.002551              0.025692   
5                                0.001256              0.030151   
6                                0.005510              0.012397   
7                                0.006501              0.021668   
8                                0.015101              0.008389   
9                                0.014074              0.013333   
10                               0.013263              0.017241   
11                               0.001328              0.022576   
12                               0.003285              0.017363   
13                               0.001741              0.016536   
14                               0.002845              0.019915   
15                               0.003501              0.016106   

housing_char                                                             \
sub_char       Electricity Electric Boiler Electricity Electric Furnace   
HousingGroupNo                                                            
1                                      NaN                     0.019006   
2                                 0.000328                     0.008320   
3                                      NaN                     0.017375   
4                                 0.000364                     0.006013   
5                                      NaN                     0.010050   
6                                 0.001377                     0.004132   
7                                      NaN                     0.010834   
8                                      NaN                     0.020134   
9                                      NaN                     0.021481   
10                                     NaN                     0.022546   
11                                0.000332                     0.007968   
12                                     NaN                     0.009385   
13                                     NaN                     0.006092   
14                                     NaN                     0.007112   
15                                     NaN                     0.010504   

housing_char                                                           ...  \
sub_char       Electricity Electric Wall Furnace Fuel Oil Fuel Boiler  ...   
HousingGroupNo                                                         ...   
1                  

In [141]:
## Combine "groups" + "House" into "Out" for export
out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out = pd.concat([out.set_index('HousingGroupNo')], axis=1, keys=['none']).join(House)
out

none  \
                                     HousingGroupName   
HousingGroupNo                                          
1               Masonry Post-1978 less than 2 stories   
2                       Masonry All Years Split Level   
3                 Masonry Post-1978 2 or more stories   
4               Masonry 1942-1978 Less than 2 Stories   
5                 Masonry 1942-1978 2 or more stories   
6                Masonry Pre-1942 less than 2 stories   
7                  Masonry Pre-1942 2 or more stories   
8                 Frame Post-1978 Less than 2 stories   
9                         Frame Post-1978 Split Level   
10                  Frame Post-1978 2 or more stories   
11                Frame 1942-1978 less than 2 stories   
12                         Frame Pre-1942 Split Level   
13                  Frame 1942-1978 2 or more stories   
14                 Frame Pre-1942 less than 2 stories   
15                   Frame Pre-1942 2 or more stories   

                                                                            \
               Non-normalized gas usage Non-normalized elec. usage   count   
HousingGroupNo                                                               
1                                  1133                      10282   684.0   
2                                  1416                       9661  9135.0   
3                                  1812                      18225   518.0   
4                                  1210                       8779  5488.0   
5                                  1427                      10549   796.0   
6                                  1504                       9299   726.0   
7                                  2044                      13272   923.0   
8                                  1054                       9353   596.0   
9                                  1168                      10445  1350.0   
10                                 1422                      13495   754.0   
11                                 1176                       9213  3012.0   
12                                 1297                       9964  2131.0   
13                                 1530                      12543  1149.0   
14                                 1340                       9103   703.0   
15                                 1753                      11967  1428.0   

                                        HVAC Cooling Type                      \
               median sqft    mean sqft        Central AC Heat Pump      None   
HousingGroupNo                                                                  
1                   2176.0  2386.103801          0.972222  0.013158  0.004386   
2                   1690.0  1999.694691          0.835796  0.005802  0.010290   
3                   3301.0  4054.416988          0.961390  0.027027  0.007722   
4                   1690.0  1680.132653          0.863338  0.003462  0.011844   
5                   2176.0  2388.954774          0.868090  0.001256  0.012563   
6                   1220.0  1460.060606          0.650138  0.005510  0.009642   
7                   2176.0  2549.022752          0.618635  0.006501  0.005417   
8                   2176.0  2243.300336          0.953020  0.018456  0.011745   
9                   2176.0  3009.053333          0.958519  0.019259  0.006667   
10                  2663.0  3614.343501          0.962865  0.019894  0.002653   
11                  1690.0  1690.260624          0.885790  0.003652  0.014608   
12                  1690.0  2019.074613          0.616612  0.003285  0.006570   
13                  1690.0  2265.897302          0.874674  0.001741  0.017406   
14                  1220.0  1402.300142          0.624467  0.002845  0.007112   
15                  2176.0  2322.710784          0.612745  0.003501  0.006303   

                          ...       Insulation Wall  \
                 Room AC  ... CMU, 6-in Hollow, R-7   
HousingGroupNo            ...                   

### Write output to folder

In [142]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'buildstock_prototype_results.csv'), index=True)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_11
